In [ ]:
import keras, os
import numpy as np
from models import model1

### Hyperparameters

In [2]:
n_timesteps = 30
n_securities = 20
input_shape=((n_timesteps-1, n_securities))
embed_dim = 384
n_heads = 6
n_layers = 2
ff_dim = embed_dim*4

batch_size = 64
epochs = 200
lr = 1e-4

### Data Pipeline

In [3]:
def preprocess_dataset(data, n_timesteps):
    n_examples = data.shape[0]-n_timesteps
    ds = []
    for idx in range(n_examples):
        X = data[idx:idx+n_timesteps]
        ds.append(X)
    return np.array(ds) # (B, timesteps, n_securities)

class dataset(keras.utils.PyDataset):

    def load(self, path):
        y = np.load(path)
        y = 0.1*y/np.std(y)
        return y    

    def __init__(
            self,
            batch_size,
            n_timesteps,
            data,           
            **kwargs       
            ):

        super().__init__(**kwargs)
        self.batch_size = batch_size
        self.n_timesteps = n_timesteps
        self.data = data
    
    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        X = self.data[idx:idx+self.batch_size,:-1]     # (B, n_timesteps-1, n_securities)
        Y = self.data[idx:idx+self.batch_size,-1]      # (B, n_securities)
        return X, Y

### Load Dataset

In [4]:
P = np.random.randn(1024, n_securities)

# Create dataset splits
ds = preprocess_dataset(P, n_timesteps)
n_examples = ds.shape[0]
train_ds = dataset(batch_size, n_timesteps, ds[:n_examples*80//10])
val_ds = dataset(batch_size, n_timesteps, ds[:n_examples*20//100])

### Build Model

In [5]:
def compile(model, checkpoint, lr):
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss=keras.losses.MeanSquaredError(),
        )    
    if checkpoint is not None:
        if os.path.exists(checkpoint):
            print(f'Loaded model checkpoint "{checkpoint}"')
            model = model.load_weights(checkpoint)
        else:
            print(f'New model "{checkpoint}"')

In [ ]:
m1 = model1(input_shape, n_layers, embed_dim, n_heads, ff_dim)
m1.summary()

m1_checkpoint = './checkpoints/model1.weights.h5'
compile(m1, m1_checkpoint, lr)

### Train Model

In [ ]:
m1.fit( 
    x=train_ds, epochs=epochs, validation_data=val_ds,
    callbacks=
    [
        keras.callbacks.ModelCheckpoint(filepath=m1_checkpoint, save_weights_only=True, monitor="val_loss", mode="min", save_best_only=True),
    ])